In [88]:
# The code was removed by Watson Studio for sharing.

# Final capstone project

## Tehran venues analysis

## 1. Introduction

### 1.1 Background

Tehran is a very big city in terms of the number of districts and population. Tehran has over 22 districts with a population of 12 million people(2018) which makes it the second most populated city in the middle east after Istanbul. A big city like Tehran has lot’s of neighborhoods and many venues.

### 1.2 Problem

Unfortunately, a big city like Tehran doesn’t have a good analysis of their venues. For example, a tourist doesn’t know the best places in Tehran based on each neighborhood so In this project we are going to cluster each neighborhood in 22 districts of Tehran based on their top 10 venues in each neighborhood

### 1.3 Interest

The output of this project can be very helpful for tourists or anyone who is interested in finding the best venues of Tehran based on their neigborhoods.

## 2. Data acquisition

Surprisingly, Tehran doesn’t have a clean database for data scientists. As a data scientist you have to pretty much collect everything you need yourself. So my first step would be to collect neighborhoods data from here and put each one in a row of my Dataframe. After this step I will collect the coordinates for my neighborhoods to plot my data on a map. Coordinates will be collected by Geopy library. For the Final step, I will get my venues data based on each neighborhood using Foursquare API which surprisingly has a valuable database for the city of Tehran.

## 3.Data analysis explanation

For start first we first import our package:

In [89]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium

print('Libraries imported.')

Libraries imported.


### 3.1 Data gathering

Now we need to start our data gathering process. As explained above, since there is no csv file to convert data frame and wrangle the data we will create our dataframe directly using pandas library. The dataframe we are looking to create consists of columns <i> District, neighborhood, lantitude, logtitude </i>. To have the first two columns which are <b>District</b> and <b>neighborhood</b> we will get what we need from <a href="https://en.wikipedia.org/wiki/Template:Main_neighborhoods_of_Tehran">here</a> and we will put each neighborhood to it's district. Since the data in this page is in Farsi and there is no table, data scraping tools can't help us to retrieve data from this page.

In [90]:
dg = [
    ('North', 'Aghdasieh'),
    ('North', 'Lavizan'),
    ('North', 'Ajodanieh'),
    ('North', 'Darakeh'),
    ('North', 'Darband'),
    ('North', 'Darus'),
    ('North', 'Mirdamad'),
    ('North', 'Doulat'),
    ('North', 'Ekhtiarieh'),
    ('North', 'Elahieh'),
    ('North', 'Farmanieh'),
    ('North', 'Gheytarieh'),
    ('North', 'Gholhak'),
    ('North', 'Jamaran'),
    ('North', 'Jordan'),
    ('North', 'Kamranieh'),
    ('North', 'Mahmoodieh'),
    ('North', 'Mehran'),
    ('North', 'Niavaran'),
    ('North', 'Pasdaran'),
    ('North', 'Shemiran'),
    ('North', 'Tajrish'),
    ('North', 'Vanak'),
    ('North', 'Valiasr'),
    ('North', 'Velenjak'),
    ('North', 'Zafaraniyeh'),
    ('North', 'North Shahran'),
    ('West', 'Ekbatan'),
    ('West', 'Shahrak Apadana'),
    ('West', 'Bagh Feyz'),
    ('West', 'Farahzad'),
    ('West', 'Gisha'),
    ('West', 'Jannat Abad'),
    ('West', 'Punak'),
    ('West', "Sa'adat Abad"),
    ('West', 'Sadeghiyeh'),
    ('West', 'Shahrak Gharb'),
    ('West', 'Shahran'),
    ('West', 'Shahrara'),
    ('West', 'Shahr-e Ziba'),
    ('West', 'Tarasht'),
    ('West', 'Tehransar'),
    ('Central', 'Abbas Abad'),
    ('Central', 'Amir Abad'),
    ('Central', 'Baharestan'),
    ('Central', 'Meydan-e Enghelab-e Eslami'),
    ('Central', 'Bazar'),
    ('Central', 'Hasan Abad'),
    ('Central', 'Jomhuri'),
    ('Central', 'Keshavarz blvd'),
    ('Central', 'Park-e Shahr'),
    ('Central', 'Seyed Khandan'),
    ('Central', 'Toopkhaneh'),
    ('Central', 'Tehran No'),
    ('Central', 'Haft-e-Tir'),
    ('East', 'Afsariyeh'),
    ('East', 'Lavizan'),
    ('East', 'Narmak'),
    ('East', 'Tehranpars'),
    ('East', 'Tehranno'),
    ('East', 'Piroozi'),
    ('South', 'Gomrok'),
    ('South', 'Javadiyeh'),
    ('South', 'Khavaran'),
    ('South', 'Navvab'),
    ('South', 'Nazi Abad'),
    ('South', 'Rey'),
    ('South', 'Yaft Abad')
]
data = pd.DataFrame(dg, columns=('City side', 'Neighborhood'))
data.head()

,City side,Neighborhood
0,North,Aghdasieh
1,North,Lavizan
2,North,Ajodanieh
3,North,Darakeh
4,North,Darband


In [91]:
print('Tehran has {} unique neighborhoods.'.format(len(data['Neighborhood'].unique())))

Tehran has 67 unique neighborhoods.


### Now let's find lantitude and longtitude for each neighborhood using geopy.

In [92]:
lan, lon = [], []
for neighborhood in data['Neighborhood']:
    address = str(neighborhood) + ', Tehran'
    
    geolocator = Nominatim(user_agent="ir_explorer")
    location = geolocator.geocode(address)
    if location == None:
        print('No location for ' + address)
        lan.append(neighborhood+'Q')
        lon.append(neighborhood+'E')
        continue
    else:
        latitude = location.latitude
        longitude = location.longitude
    
        lan.append(latitude)
        lon.append(longitude)
    
    print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))
    
data['latitude'] = lan
data['longitude'] = lon

The geograpical coordinate of Aghdasieh, Tehran are 35.5938365, 51.4444056.
The geograpical coordinate of Lavizan, Tehran are 35.7770552, 51.5021499.
No location for Ajodanieh, Tehran
The geograpical coordinate of Darakeh, Tehran are 35.8043457, 51.3827097.
The geograpical coordinate of Darband, Tehran are 35.8135171, 51.429482.
The geograpical coordinate of Darus, Tehran are 35.7727353, 51.4566868.
The geograpical coordinate of Mirdamad, Tehran are 35.759387, 51.43502.
The geograpical coordinate of Doulat, Tehran are 35.7046285, 51.3607125.
The geograpical coordinate of Ekhtiarieh, Tehran are 35.7863456, 51.4611775.
The geograpical coordinate of Elahieh, Tehran are 35.7898612, 51.4262023.
The geograpical coordinate of Farmanieh, Tehran are 35.80325, 51.4594829.
The geograpical coordinate of Gheytarieh, Tehran are 35.79030505, 51.4449767570462.
The geograpical coordinate of Gholhak, Tehran are 35.7731769, 51.4442124.
The geograpical coordinate of Jamaran, Tehran are 35.8198534, 51.4583

As you can See some geopy can't give coordinates for some locations which is because of the old name of some neighborhoods. So we will explicitly give coordinates for those locations inside our for loop and place it in our dataframe.

In [93]:
data.replace(to_replace='AjodaniehQ', value=35.8078, inplace=True)
data.replace(to_replace='AjodaniehE', value=51.4836, inplace=True)
data.replace(to_replace='ToopkhanehQ', value=35.68578, inplace=True)
data.replace(to_replace='ToopkhanehE', value=51.42018, inplace=True)
data.replace(to_replace='TehrannoQ', value=35.70979, inplace=True)
data.replace(to_replace='TehrannoE', value=51.49646, inplace=True)
data.replace(to_replace=35.5614809, value=35.70783, inplace=True)
data.replace(to_replace=51.1724907, value=51.38808, inplace=True)

In [94]:
data.head()

,City side,Neighborhood,latitude,longitude
0,North,Aghdasieh,35.593837,51.444406
1,North,Lavizan,35.777055,51.502150
2,North,Ajodanieh,35.807800,51.483600
3,North,Darakeh,35.804346,51.382710
4,North,Darband,35.813517,51.429482


### Now that we have gathered Each column we needed in a dataframe, we can plot neighbourhoods on a map just to check that we have included everything we wanted from the city of tehran.

In [95]:
# create map of Manhattan using latitude and longitude values
map_tehran = folium.Map(location=[35.6981, 51.3836], zoom_start=11)

# add markers to map
for lat, lng, label in zip(data['latitude'], data['longitude'], data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tehran)  
    
map_tehran

## Okay, Now that we are sure about our geographical data (City side, latitude, longitude) we can proceed to forsquare api to get our venues data from it's database.

### Define Foursquare Credentials and Version

In [96]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: SXI334UDPMF3ETEFHTUZXJZSR43PR3RXZEQSVXYI0QTV0NJ2
CLIENT_SECRET:0NO1OUS5BJCI1TFUU0WWA1QFJD4ER1U1MFJD0CAAHT10YBVL


Let's Just explore one neighborhood for start.

In [97]:
data.loc[6, 'Neighborhood']

'Mirdamad'

In [98]:
neighborhood_latitude = data.loc[6, 'latitude'] # neighborhood latitude value
neighborhood_longitude = data.loc[6, 'longitude'] # neighborhood longitude value

neighborhood_name = data.loc[6, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Mirdamad are 35.759387, 51.43502.


In [99]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=SXI334UDPMF3ETEFHTUZXJZSR43PR3RXZEQSVXYI0QTV0NJ2&client_secret=0NO1OUS5BJCI1TFUU0WWA1QFJD4ER1U1MFJD0CAAHT10YBVL&v=20180605&ll=35.759387,51.43502&radius=500&limit=100'

In [100]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d6ce489a4b51b002c8bd632'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Minṭaqa 3',
  'headerFullLocation': 'Minṭaqa 3, Tehrān',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 34,
  'suggestedBounds': {'ne': {'lat': 35.7638870045, 'lng': 51.440555086702204},
   'sw': {'lat': 35.754886995499994, 'lng': 51.4294849132978}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5477387b498e1ceac8d12f9b',
       'name': 'Bahar Café & Gallery | کافه گالری بهار (کافه گالری بهار)',
       'location': {'address': 'Mirdamad Blvd.',
        'crossStreet': 'Pasargad Shopping centre - beside Alqadir Mosque',
        'lat': 35.759468152438515,

#### function that extracts the category of the venue

In [101]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [102]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Bahar Café & Gallery | کافه گالری بهار (کافه گ...,Café,35.759468,51.437495
1,Lamiz Coffee | لمیز کافی (لمیز کافی),Coffee Shop,35.759974,51.434052
2,بهارخانه | Baharkhaneh,Arts & Crafts Store,35.759940,51.437265
3,استخر قصر موج,Pool,35.758798,51.431316
4,Book City | شهر کتاب آرین (شهر کتاب آرین),Bookstore,35.760220,51.430536


In [103]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

34 venues were returned by Foursquare.


Now lets do this process for other neighborhoods aswell

In [104]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [105]:
tehran_venues = getNearbyVenues(names=data['Neighborhood'],
                                   latitudes=data['latitude'],
                                   longitudes=data['longitude']
                                  )

Aghdasieh
Lavizan
Ajodanieh
Darakeh
Darband
Darus
Mirdamad
Doulat
Ekhtiarieh
Elahieh
Farmanieh
Gheytarieh
Gholhak
Jamaran
Jordan
Kamranieh
Mahmoodieh
Mehran
Niavaran
Pasdaran
Shemiran
Tajrish
Vanak
Valiasr
Velenjak
Zafaraniyeh
North Shahran
Ekbatan
Shahrak Apadana
Bagh Feyz
Farahzad
Gisha
Jannat Abad
Punak
Sa'adat Abad
Sadeghiyeh
Shahrak Gharb
Shahran
Shahrara
Shahr-e Ziba
Tarasht
Tehransar
Abbas Abad
Amir Abad
Baharestan
Meydan-e Enghelab-e Eslami
Bazar
Hasan Abad
Jomhuri
Keshavarz blvd
Park-e Shahr
Seyed Khandan
Toopkhaneh
Tehran No
Haft-e-Tir
Afsariyeh
Lavizan
Narmak
Tehranpars
Tehranno
Piroozi
Gomrok
Javadiyeh
Khavaran
Navvab
Nazi Abad
Rey
Yaft Abad


In [106]:
print(tehran_venues.shape)
tehran_venues.head()

(1072, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aghdasieh,35.593837,51.444406,Fina Food | فینا فود,35.593667,51.442405,Fast Food Restaurant
1,Aghdasieh,35.593837,51.444406,زمين چمن مطهري,35.591102,51.446235,Football Stadium
2,Aghdasieh,35.593837,51.444406,فست فود آرادArad Fastfood,35.593630,51.449789,Fast Food Restaurant
3,Ajodanieh,35.807800,51.483600,R&A coffee shop,35.805885,51.484921,Coffee Shop
4,Ajodanieh,35.807800,51.483600,Royal Aghdasiye Fitness Center,35.805618,51.479701,Gym / Fitness Center


As you can see we have found 1072 venues based on forsquare data which is fairly not so well for a city like tehran.

Let's check how many venues were returned for each neighborhood

In [107]:
tehran_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbas Abad,12,12,12,12,12,12
Afsariyeh,6,6,6,6,6,6
Aghdasieh,3,3,3,3,3,3
Ajodanieh,11,11,11,11,11,11
Amir Abad,7,7,7,7,7,7
Bagh Feyz,6,6,6,6,6,6
Baharestan,24,24,24,24,24,24
Bazar,23,23,23,23,23,23
Darakeh,21,21,21,21,21,21


In [108]:
print('There are {} uniques categories.'.format(len(tehran_venues['Venue Category'].unique())))

There are 167 uniques categories.


# Let's Analyze Each Neighborhood

In [109]:
# one hot encoding
tehran_onehot = pd.get_dummies(tehran_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tehran_onehot['Neighborhood'] = tehran_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tehran_onehot.columns[-1]] + list(tehran_onehot.columns[:-1])
tehran_onehot = tehran_onehot[fixed_columns]

tehran_onehot.head()

,Yoga Studio,Accessories Store,Airport Terminal,Amphitheater,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Ash and Haleem Place,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bistro,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Camera Store,Campground,Candy Store,Carpet Store,Cheese Shop,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cultural Center,Czech Restaurant,Dairy Store,Department Store,Design Studio,Dessert Shop,Diner,Dizi Place,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jegaraki,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Leather Goods Store,Lounge,Market,Massage Studio,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Mosque,Movie Theater,Multiplex,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Public Art,Recreation Center,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shrine,Snack Place,Soccer Field,Soup Place,Sporting Goods Shop,Sports Club,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tabbakhi,Tailor Shop,Taxi Stand,Tea Room,Tennis Court,Tennis Stadium,Theater,Toy / Game Store,Trail,Tree,Veterinarian,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aghdasieh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aghdasieh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aghdasieh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ajodanieh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ajodanieh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [110]:
tehran_onehot.shape

(1072, 167)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [111]:
tehran_grouped = tehran_onehot.groupby('Neighborhood').mean().reset_index()
tehran_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport Terminal,Amphitheater,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Ash and Haleem Place,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Bistro,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Camera Store,Campground,Candy Store,Carpet Store,Cheese Shop,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cultural Center,Czech Restaurant,Dairy Store,Department Store,Design Studio,Dessert Shop,Diner,Dizi Place,Donut Shop,Drugstore,Dry Cleaner,Electronics Store,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,IT Services,Ice Cream Shop,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jegaraki,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Leather Goods Store,Lounge,Market,Massage Studio,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Mosque,Movie Theater,Multiplex,Music Store,Music Venue,Nail Salon,National Park,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Perfume Shop,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Public Art,Recreation Center,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shrine,Snack Place,Soccer Field,Soup Place,Sporting Goods Shop,Sports Club,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tabbakhi,Tailor Shop,Taxi Stand,Tea Room,Tennis Court,Tennis Stadium,Theater,Toy / Game Store,Trail,Tree,Veterinarian,Wings Joint,Women's Store
0,Abbas Abad,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00000,0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.083333
1,Afsariyeh,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [112]:
tehran_grouped.shape

(66, 167)

In [113]:
num_top_venues = 5

for hood in tehran_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tehran_grouped[tehran_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbas Abad----
                venue  freq
0  Persian Restaurant  0.25
1               Plaza  0.17
2       Women's Store  0.08
3          Astrologer  0.08
4              Market  0.08


----Afsariyeh----
              venue  freq
0             Plaza  0.17
1       Sports Club  0.17
2         Bookstore  0.17
3             Diner  0.17
4  Kebab Restaurant  0.17


----Aghdasieh----
                  venue  freq
0  Fast Food Restaurant  0.67
1      Football Stadium  0.33
2           Yoga Studio  0.00
3            Nail Salon  0.00
4   Moroccan Restaurant  0.00


----Ajodanieh----
                  venue  freq
0                   Gym  0.18
1           Pizza Place  0.09
2           Coffee Shop  0.09
3                  Café  0.09
4  Fast Food Restaurant  0.09


----Amir Abad----
                     venue  freq
0              Sports Club  0.29
1       Athletics & Sports  0.14
2  Health & Beauty Service  0.14
3             Burger Joint  0.14
4             Soccer Field  0.14


----Bagh Feyz----

#### Let's put that into a *pandas* dataframe

In [114]:
#### Let's put that into a *pandas* dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [115]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tehran_grouped['Neighborhood']

for ind in np.arange(tehran_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tehran_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbas Abad,Persian Restaurant,Plaza,Tailor Shop,Costume Shop,Carpet Store,Metro Station,Market,Astrologer,Women's Store,Furniture / Home Store
1,Afsariyeh,Bookstore,Plaza,Sports Club,Diner,Kebab Restaurant,Supermarket,Drugstore,Fast Food Restaurant,Farm,Falafel Restaurant
2,Aghdasieh,Fast Food Restaurant,Football Stadium,Women's Store,Donut Shop,Farm,Falafel Restaurant,Fabric Shop,Electronics Store,Dry Cleaner,Drugstore
3,Ajodanieh,Gym,Fast Food Restaurant,Park,Gym / Fitness Center,Pizza Place,Coffee Shop,Japanese Restaurant,Café,Market,Auto Garage
4,Amir Abad,Sports Club,Soccer Field,IT Services,Burger Joint,Athletics & Sports,Health & Beauty Service,Cultural Center,Drugstore,Farm,Creperie


### Clustering the data

In [116]:
# set number of clusters
kclusters = 5

tehran_grouped_clustering = tehran_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tehran_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 0, 0, 0, 0, 0, 2, 2], dtype=int32)

In [117]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

teh_merged = data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
teh_merged = teh_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

teh_merged.head() # check the last columns!

,City side,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North,Aghdasieh,35.593837,51.444406,4.0,Fast Food Restaurant,Football Stadium,Women's Store,Donut Shop,Farm,Falafel Restaurant,Fabric Shop,Electronics Store,Dry Cleaner,Drugstore
1,North,Lavizan,35.777055,51.502150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,North,Ajodanieh,35.807800,51.483600,0.0,Gym,Fast Food Restaurant,Park,Gym / Fitness Center,Pizza Place,Coffee Shop,Japanese Restaurant,Café,Market,Auto Garage
3,North,Darakeh,35.804346,51.382710,2.0,Persian Restaurant,Café,Restaurant,Tennis Court,Athletics & Sports,Hookah Bar,Trail,Supermarket,Sports Club,Diner
4,North,Darband,35.813517,51.429482,2.0,History Museum,Garden,Café,Burger Joint,Supermarket,Steakhouse,Art Museum,Department Store,Gaming Cafe,River


At the end let's just map our clusters!

In [128]:
# create map
map_clusters = folium.Map(location=[35.6981, 51.3836], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(teh_merged['latitude'], teh_merged['longitude'], teh_merged['Neighborhood'], teh_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# We have successfully clustered our data!